# Bermudan Option Pricing via a Pure Dual Approach
Implementation based on the paper "A pure dual approach for hedging Bermudan options" by Alfonsi, Kebaier, and Lelong.

In [11]:
import numpy as np
from sklearn.linear_model import LinearRegression

def simulate_gbm(S0, r, sigma, T, N, M):
    dt = T / N
    paths = np.zeros((M, N+1))
    paths[:, 0] = S0
    for t in range(1, N+1):
        paths[:, t] = paths[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * np.random.randn(M))
    return paths

def compute_martingale_approximation(S0, K, r, sigma, T, N, M, basis_degree=4):
    # Simulate paths
    paths = simulate_gbm(S0, r, sigma, T, N, M)
    Z = np.maximum(K - paths, 0)  # Put option payoff

    delta_M = np.zeros((M, N+1))      # Martingale increments
    deltas = np.zeros((N+1,))         # Estimated hedge ratios (one per time step)

    for n in range(N-1, -1, -1):
        max_values = np.zeros(M)
        if n+1 <= N:
            current_max = Z[:, n+1].copy()  # j = n+1 case
            for j in range(n+2, N+1):
                sum_delta = np.sum(delta_M[:, n+2:j+1], axis=1)
                current_val = Z[:, j] - sum_delta
                current_max = np.maximum(current_max, current_val)
            max_values = current_max

        X_next = paths[:, n+1]
        basis_next = np.column_stack([X_next**k for k in range(1, basis_degree+1)])

        X_current = paths[:, n]
        basis_current = np.column_stack([X_current**k for k in range(0, basis_degree+1)])

        # Construct orthogonal basis (innovation) for martingale approximation
        adjusted_basis = np.zeros_like(basis_next)
        for k in range(basis_next.shape[1]):
            model = LinearRegression()
            model.fit(basis_current, basis_next[:, k])
            pred = model.predict(basis_current)
            adjusted_basis[:, k] = basis_next[:, k] - pred

        # Regression for martingale increment
        residuals = max_values - Z[:, n]
        model = LinearRegression(fit_intercept=False)
        model.fit(adjusted_basis, residuals)
        delta_M[:, n+1] = model.predict(adjusted_basis)

        # Regression for delta hedge estimate (on original basis)
        model_delta = LinearRegression(fit_intercept=False)
        model_delta.fit(basis_current, residuals)

        # Get coefficient corresponding to S_n (assuming basis is [1, S_n, S_n^2, ...])
        if basis_current.shape[1] >= 2:
            deltas[n] = model_delta.coef_[1]  # delta w.r.t. S_n
        else:
            deltas[n] = 0.0  # fallback if degree is too low

    # Compute martingale path
    M_martingale = np.cumsum(delta_M, axis=1)

    return M_martingale, delta_M, paths, Z, deltas


In [12]:
def compute_lower_bound(paths, K, r, T, N, basis_degree=4):
    dt = T / N
    discount = np.exp(-r * dt)
    M = paths.shape[0]
    cashflows = np.maximum(K - paths[:, -1], 0)  # Put option payoff at maturity

    for t in range(N-1, 0, -1):
        itm = np.where(paths[:, t] < K)[0]  # In-the-money paths
        X = paths[itm, t]
        Y = cashflows[itm] * discount

        # Regression on polynomial basis
        basis = np.column_stack([X**d for d in range(basis_degree + 1)])
        model = LinearRegression()
        model.fit(basis, Y)
        continuation = model.predict(basis)

        exercise = K - X > continuation
        cashflows[itm[exercise]] = K - X[exercise]
        cashflows *= discount  # Discount all cashflows one step back

    return np.mean(cashflows)

In [13]:
# Example parameters
M, T, n = 100_000, 1.0, 100
r, vol, S0 = 0.06, 0.2, 36.0
K = 40
dt   = T / n

In [17]:
for k in range(35,46):# The true adjusted payoff at time 0 is the max over Z_j - sum of delta_M from 1 to j
    M_martingale, delta_M, paths, Z , deltas= compute_martingale_approximation(
        S0, k, r, vol, T, n, M
    )
    upper_bounds = np.zeros(M)
    for j in range(1, n+1):
        adjusted = Z[:, j] - np.sum(delta_M[:, 1:j+1], axis=1)
        upper_bounds = np.maximum(upper_bounds, adjusted)

    upper_bound_estimate = np.mean(upper_bounds)
    lower_bound_estimate = compute_lower_bound(paths, k, r, T, n, basis_degree=4)
    U_0 = np.max(Z - M_martingale, axis=1)
    bermudan_price = np.exp(-r * T) * np.mean(U_0)


    t = np.linspace(0, T, n+1)
    discounted_Z = Z * np.exp(-r * t)  

    max_values = np.max(discounted_Z - M_martingale, axis=1)

    price_estimate = np.mean(max_values)

    
    
    print(f"for K = {k} LB : {lower_bound_estimate:.4f}, UB : {upper_bound_estimate:.4f}, estimated price : {price_estimate:.4f}")
    print("the associated strategy is :")
    for i in range(n+1):
        print(f"{i:9d} | {deltas[i]:.4f}")

for K = 35 LB : 1.6634, UB : 1.9496, estimated price : 1.8624
the associated strategy is :
        0 | 0.0000
        1 | -83.1814
        2 | -53.4523
        3 | -36.8395
        4 | -27.9768
        5 | -22.5638
        6 | -18.7107
        7 | -16.0607
        8 | -93.4413
        9 | -84.0701
       10 | -82.0708
       11 | -62.4948
       12 | -52.4671
       13 | -48.4332
       14 | -39.4659
       15 | -37.4524
       16 | -29.6804
       17 | -24.9165
       18 | -21.6263
       19 | -17.9695
       20 | -21.6315
       21 | -23.4938
       22 | -20.8975
       23 | -19.0639
       24 | -17.8648
       25 | -13.8899
       26 | -12.1271
       27 | -11.3169
       28 | -9.3725
       29 | -7.4156
       30 | -7.1378
       31 | -4.2380
       32 | -3.5280
       33 | -3.9295
       34 | -3.6188
       35 | -4.8872
       36 | -4.3889
       37 | -4.5089
       38 | -4.5867
       39 | -4.2654
       40 | -3.9086
       41 | -3.3262
       42 | -2.7389
       43 | -2.3841
   

In [6]:


# Calculate discount factors for each time step
t = np.linspace(0, T, n+1)
discounted_Z = Z * np.exp(-r * t)  # Discount each payoff to present value

# Compute the maximum of (discounted_Z - M_martingale) for each path
max_values = np.max(discounted_Z - M_martingale, axis=1)

# Average the maxima to get the price estimate
price_estimate = np.mean(max_values)

print(f"Estimated American put price: {price_estimate:.4f}")

Estimated American put price: 9.0263
